In [2]:
import tensorflow as tf
from tensorflow import keras


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [38]:
dataset_train = pd.read_csv('output.csv')

training_set = dataset_train.iloc[:, 0:3].values

In [39]:

print(training_set)

[['T06' '2016-01-08T23:10:00+00:00' 1634.3]
 ['T06' '2016-04-19T12:20:00+00:00' 1796.1]
 ['T01' '2016-01-08T23:10:00+00:00' 1657.5]
 ...
 ['T01' '2016-10-22T10:50:00+00:00' 1672.4]
 ['T06' '2016-10-22T11:20:00+00:00' 1659.8]
 ['T01' '2016-10-22T11:20:00+00:00' 1668.5]]


In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [7]:
X_train = []
y_train = []
for i in range(60, 2035):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [8]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
62/62 [==============================] - 9s 52ms/step - loss: 0.1014
Epoch 2/100
62/62 [==============================] - 3s 51ms/step - loss: 0.0802
Epoch 3/100
62/62 [==============================] - 3s 51ms/step - loss: 0.0748
Epoch 4/100
62/62 [==============================] - 3s 55ms/step - loss: 0.0723
Epoch 5/100
62/62 [==============================] - 3s 55ms/step - loss: 0.0720
Epoch 6/100
62/62 [==============================] - 3s 54ms/step - loss: 0.0693
Epoch 7/100
62/62 [==============================] - 3s 56ms/step - loss: 0.0690
Epoch 8/100
62/62 [==============================] - 3s 55ms/step - loss: 0.0671
Epoch 9/100
62/62 [==============================] - 3s 55ms/step - loss: 0.0678
Epoch 10/100
62/62 [==============================] - 3s 56ms/step - loss: 0.0661
Epoch 11/100
62/62 [==============================] - 3s 55ms/step - loss: 0.0660
Epoch 12/100
62/62 [==============================] - 3s 55ms/step - loss: 0.0652
Epoch 13/100
62/62 [=====

In [24]:
dataset_test = pd.read_csv('output.csv')
dataset_total = pd.read_csv('output.csv')

dataset_test.drop(['Turbine_ID'], axis = 1,inplace=True)
dataset_test.drop(['Timestamp'], axis = 1,inplace=True)
real_stock_price = dataset_test.iloc[:, 2:3].values


#dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_test[len(dataset_test) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 76):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)


In [30]:
# Visualising the results
plt.plot(dataset_test.loc['Turbine_ID'],dataset_test.values, color = 'red', label = 'Real TESLA Stock Price')
plt.plot(dataset_test.loc[800:, 'Turbine_ID'],predicted_stock_price, color = 'blue', label = 'Predicted TESLA Stock Price')
plt.xticks(np.arange(0,459,50))
plt.title('TESLA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TESLA Stock Price')
plt.legend()
plt.show()

KeyError: 'Turbine_ID'